# Installing GATK + notes
    This page will be a reference for installation, different tutorials and how to call polyploids

  #### Get GATK jar and .yml files here:
  https://github.com/broadinstitute/gatk/releases
  
  wget .zip file to a src directory, unzip, run conda install below in that direcoty
  
  #### directions for conda install here:
  https://gatk.broadinstitute.org/hc/en-us/articles/360035889851--How-to-Install-and-use-Conda-for-GATK4
  
  #### vivaswat notes
  https://microcollaborative.atlassian.net/wiki/spaces/BL/pages/73957377/Bioinformatics+for+polyploids
  
  
  
  #### IDEA: 
  
  - call snps with GATK haplotype caller. 
  - run one per sample with correct ploidy into gvcf
  - then use GenotypeGVCF to call snps across all gvcfs, no need to tell ploidy.  
  

In [3]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [139]:
root = "/data/gpfs/assoc/denovo/tfaske/sagebrush/"

In [140]:
snp_dir = os.path.join(root,'SNPcall')

In [141]:
bwa_dir = os.path.join(snp_dir,'bwa')

In [14]:
cd $bwa_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/bwa


In [15]:
bam_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    bam_files.append(x)
bam_files = sorted(bam_files)

In [16]:
len(bam_files), bam_files[0]

(700,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/bwa/AT2_BB_10_sorted.bam')

In [83]:
samtools = "samtools"
bcftools = "bcftools"
java = "java"
perl = "perl"
#perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.18/bin/perl"
gatk = '/data/gpfs/assoc/parchmanlab/tfaske/src/gatk4/gatk'

In [45]:
assembly = "/data/gpfs/assoc/denovo/tfaske/sagebrush/assembly/GCA_021014915.1/GCA_021014915.1_ASM2101491v1.fna"


#need to index with picard and samtools
#picard CreateSequenceDictionary R=GCA_021014915.1_ASM2101491v1.fna O=GCA_021014915.1_ASM2101491v1.dict


In [20]:
!$samtools faidx {assembly}

### SKIP: too big. Calculates coverage from bam_files
also outputs a file called "bam_coverage.csv" in the bam_coverage folder  

this will make a file for each sorted.bam and then consolidate in order to run efficiently

first run to get size and remove empties: ls --sort=size -lh *sorted_bam 

## SNP CALL with GATK

see IDEAS for workflow. Run with a test batch of 12 with mixed ploidy to see

In [142]:
testbwa_dir = os.path.join(snp_dir,'testbwa')

In [143]:
cd $testbwa_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/testbwa


In [144]:
bam_files = []
files = !find . -type f -name '*sorted.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    bam_files.append(x)
bam_files = sorted(bam_files)

In [145]:
len(bam_files), bam_files[0]

(12,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/testbwa/AT2_EC_53_sorted.bam')

In [146]:
cd $snp_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall


In [43]:
!mkdir testgatk

In [44]:
!mkdir testgatk/shdir

In [188]:
testgatk_dir = os.path.join(snp_dir,'testgatk')

In [189]:
 #gatk --java-options "-Xmx4g" HaplotypeCaller  \
   #-R Homo_sapiens_assembly38.fasta \
   #-I input.bam \
   #-O output.g.vcf.gz \
   #-ploidy X \ 
   #-ERC GVCF \
    

def run_gatk(args):
    import os, multiprocessing, socket
    gatk, assembly, bam, out_dir = args
    ID = bam.split('/')[9] ### need to change this to match your ID 
    ID = ID.split('_sorted.bam')[0] ### This too
    ploidy = ID.split('_')[0]
    ploidy = re.sub(r'\D','',ploidy)
    #print(ID)
    #print(ploidy)
    #print(out_dir)
    cmd = r'%s --java-options "-Xmx16g" HaplotypeCaller -R %s -I %s -ploidy %s -O %s/%s.g.vcf.gz -ERC GVCF' % (gatk,assembly,bam,ploidy,out_dir,ID)
    return  cmd 

In [190]:
### creates a list of commands for gatk for each bam
res_gatk = []
for b in bam_files:
    res = run_gatk((gatk, assembly, b, testgatk_dir))
    res_gatk.append(res)

In [191]:
len(res_gatk),res_gatk[0]

(12,
 '/data/gpfs/assoc/parchmanlab/tfaske/src/gatk4/gatk --java-options "-Xmx16g" HaplotypeCaller -R /data/gpfs/assoc/denovo/tfaske/sagebrush/assembly/GCA_021014915.1/GCA_021014915.1_ASM2101491v1.fna -I /data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/testbwa/AT2_EC_53_sorted.bam -ploidy 2 -O /data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/testgatk/AT2_EC_53.g.vcf.gz -ERC GVCF')

#### Below selects options for slurm submission and is a function for creating a slurm script per fastq

In [192]:
cd $root

/data/gpfs/assoc/denovo/tfaske/sagebrush


In [193]:
bam_ID = [b.split('/')[9].split('_sorted.bam')[0] for b in bam_files]

In [194]:
len(bam_ID), bam_ID[0]

(12, 'AT2_EC_53')

In [195]:
### select options for slurm submission
#account = 'cpu-s5-denovo-0'
#partition = 'cpu-core-0'
account = 'cpu-s1-bionres-0'
partition = 'cpu-s1-bionres-0'
jobname = 'gatk'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 80000
email = 'tfaske@nevada.unr.edu'

In [196]:
def write_gatk_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,bam_ID,cmds):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    for i in range(0,len(cmds)):
        with open("SNPcall/testgatk/shdir/run_gatk_%s.sh" % (bam_ID[i]), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_gatk
#SBATCH --output output_gatk_%s.txt
#SBATCH -e error_gatk_%s.txt
#SBATCH --mail-type=FAIL
#SBATCH --mail-user=%s \n
    
%s \n\n
""" % (account,partition,time,int(cpus),int(mem_cpu),bam_ID[i],bam_ID[i],bam_ID[i],email,cmds[i]))
            

In [197]:
write_gatk_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,bam_ID,res_gatk)

#### finds all bwa slurm scripts and writes bash script to sbatch them

In [198]:
shbwa_files = []
os.chdir('{}/{}'.format(root, 'SNPcall/testgatk/shdir'))
files = !find . -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shbwa_files.append(x)
shbwa_files = sorted(shbwa_files)

In [199]:
len(shbwa_files),shbwa_files[0]

(12,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall/testgatk/shdir/run_gatk_AT2_EC_53.sh')

In [200]:
def write_bash_gatk_sh(sh_files):
    with open("SNPcall/testgatk/run_bash_gatk.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))    

In [201]:
cd $root

/data/gpfs/assoc/denovo/tfaske/sagebrush


In [202]:
write_bash_gatk_sh(shbwa_files)

# Run run_bash_gatk_sh locally
    cd /data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall 
    source activate py36
    bash run_bash_gatk.sh